In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Given parameters
S = 40                  # Spot price
K = 40                  # Strike price (ATM)
T = 5 / 12              # Time to maturity in years
q = 0.01                # Dividend yield
r = 0.025               # Risk-free rate
market_price = 2.75     # Observed market price of the call
tol = 1e-6            # Tolerance for root-finding

# Black-Scholes call price with continuous dividend yield
def bs_call_price(sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Vega for Newton's method
def vega(sigma):
    d1 = (np.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

# Define f(sigma) = BS_price(sigma) - market_price
def f(sigma):
    return bs_call_price(sigma) - market_price

# Bisection method
def bisection_method(a, b):
    steps = []
    while (b - a) / 2 > tol:
        mid = (a + b) / 2
        f_mid = f(mid)
        steps.append(mid)
        if f_mid == 0 or (b - a) / 2 < tol:
            return mid, steps
        elif f(a) * f_mid < 0:
            b = mid
        else:
            a = mid
    return (a + b) / 2, steps

# Secant method
def secant_method(x0, x1):
    steps = [x0, x1]
    while abs(x1 - x0) > tol:
        f0, f1 = f(x0), f(x1)
        x2 = x1 - f1 * (x1 - x0) / (f1 - f0)
        steps.append(x2)
        x0, x1 = x1, x2
    return x1, steps

# Newton's method
def newton_method(x0):
    steps = [x0]
    while True:
        fx = f(x0)
        v = vega(x0)
        if v == 0:
            break
        x1 = x0 - fx / v
        steps.append(x1)
        if abs(x1 - x0) < tol:
            break
        x0 = x1
    return x1, steps

# Run all methods
bisection_result, bisection_steps = bisection_method(0.0001, 1)
secant_result, secant_steps = secant_method(0.5, 0.49)
newton_result, newton_steps = newton_method(0.5)

# Combine into a single DataFrame
max_len = max(len(bisection_steps), len(secant_steps), len(newton_steps))
df = pd.DataFrame({
    "Bisection": bisection_steps + [np.nan] * (max_len - len(bisection_steps)),
    "Secant": secant_steps + [np.nan] * (max_len - len(secant_steps)),
    "Newton": newton_steps + [np.nan] * (max_len - len(newton_steps)),
})


In [12]:
print(df.to_latex(caption="Convergence of methods for implied volatility", label="tab:implied_volatility_convergence", escape=True, float_format="%.7f", index=False))

\begin{table}
\caption{Convergence of methods for implied volatility}
\label{tab:implied_volatility_convergence}
\begin{tabular}{rrr}
\toprule
Bisection & Secant & Newton \\
\midrule
0.5000500 & 0.5000000 & 0.5000000 \\
0.2500750 & 0.4900000 & 0.2556527 \\
0.3750625 & 0.2557150 & 0.2569032 \\
0.3125687 & 0.2569076 & 0.2569032 \\
0.2813219 & 0.2569032 & NaN \\
0.2656984 & 0.2569032 & NaN \\
0.2578867 & NaN & NaN \\
0.2539809 & NaN & NaN \\
0.2559338 & NaN & NaN \\
0.2569103 & NaN & NaN \\
0.2564220 & NaN & NaN \\
0.2566661 & NaN & NaN \\
0.2567882 & NaN & NaN \\
0.2568492 & NaN & NaN \\
0.2568797 & NaN & NaN \\
0.2568950 & NaN & NaN \\
0.2569026 & NaN & NaN \\
0.2569064 & NaN & NaN \\
0.2569045 & NaN & NaN \\
\bottomrule
\end{tabular}
\end{table}



In [13]:
# Newton's method result as sigma_imp
sigma_imp = newton_result

# Apply the approximation formula
numerator = np.sqrt(2 * np.pi) * (market_price - 0.5 * (r - q) * T * S)
denominator = S * np.sqrt(T) * (1 - 0.5 * (r + q) * T)
sigma_approx = numerator / denominator

# Compute the relative error
relative_error = abs(sigma_approx - sigma_imp) / sigma_imp
sigma_approx, relative_error


(0.256710246570848, 0.0007509669584858304)